### 🧠 ¿Qué es la Auto-Reflexión en RAG?
Auto-reflexión = El LLM evalúa su propia salida:
"¿Es esto claro, completo y preciso?"

#### Auto-Reflexión en RAG usando LangGraph, diseñaremos un flujo de trabajo donde el agente:

1. Genera una respuesta inicial usando el contexto recuperado
2. Reflexiona sobre esa respuesta con un paso dedicado de LLM auto-crítico
3. Si no está satisfecho, puede revisar la consulta, recuperar de nuevo, o regenerar la respuesta

In [1]:
# Importación del módulo os para interactuar con el sistema operativo (variables de entorno, rutas, etc.)
import os

# Importación de List desde typing para definir tipos de datos (listas tipadas)
from typing import List

# Importación de BaseModel desde pydantic para crear modelos de datos con validación automática
from pydantic import BaseModel

# Importación de OpenAIEmbeddings para generar embeddings (representaciones vectoriales) usando la API de OpenAI
from langchain_openai import OpenAIEmbeddings

# Importación de Document, la clase base de LangChain para representar documentos con contenido y metadatos
from langchain.schema import Document

# Importación de RecursiveCharacterTextSplitter para dividir textos largos en chunks (fragmentos) de manera recursiva
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Importación de FAISS, una biblioteca de Facebook para búsqueda eficiente de similitud en vectores
from langchain.vectorstores import FAISS

# Importación de TextLoader para cargar archivos de texto plano (.txt) como documentos
from langchain_community.document_loaders import TextLoader

# Importación de StateGraph para crear grafos de estado (flujos de trabajo con nodos y aristas)
# END es un marcador especial que indica el final del grafo
from langgraph.graph import StateGraph, END

c:\Udemy\RAGBootcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Cargar modelos LLM

# Importación del módulo os para acceder a variables de entorno
import os

# Importación de init_chat_model para inicializar modelos de chat de diferentes proveedores
from langchain.chat_models import init_chat_model

# Importación de load_dotenv para cargar variables de entorno desde un archivo .env
from dotenv import load_dotenv

# Configuración de la variable de entorno OPENAI_API_KEY con el valor obtenido del archivo .env
# Esto permite autenticarse con la API de OpenAI
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# Inicialización del modelo de lenguaje usando GPT-4o (optimizado) de OpenAI
# Este será el LLM principal para generar respuestas y reflexiones
llm=init_chat_model("openai:gpt-4o")

In [3]:
# Carga del archivo de texto "internal_docs.txt" usando TextLoader
# .load() devuelve una lista de objetos Document con el contenido del archivo
docs = TextLoader("internal_docs.txt").load()

# División de los documentos en chunks (fragmentos) más pequeños usando RecursiveCharacterTextSplitter
# chunk_size=500: cada fragmento tendrá aproximadamente 500 caracteres
# chunk_overlap=50: habrá una superposición de 50 caracteres entre fragmentos consecutivos (para mantener contexto)
chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(docs)

# Creación de un vector store (base de datos vectorial) usando FAISS
# .from_documents() toma los chunks y genera embeddings usando OpenAIEmbeddings()
# Estos embeddings permiten búsquedas de similitud semántica
vectorstore = FAISS.from_documents(chunks, OpenAIEmbeddings())

# Conversión del vector store en un retriever (recuperador)
# El retriever es la interfaz para realizar búsquedas de documentos relevantes basadas en una consulta
retriever = vectorstore.as_retriever()

In [4]:
# -------------------------
# 2. Definición del Estado
# -------------------------

# Clase que define el estado del flujo de trabajo RAG con auto-reflexión
# Hereda de BaseModel (Pydantic) para validación automática de tipos y datos
class RAGReflectionState(BaseModel):
    # question: la pregunta del usuario (tipo string, campo obligatorio)
    question: str
    
    # retrieved_docs: lista de documentos recuperados del vector store
    # Por defecto es una lista vacía []
    retrieved_docs: List[Document] = []
    
    # answer: la respuesta generada por el LLM
    # Por defecto es una cadena vacía ""
    answer: str = ""
    
    # reflection: la evaluación/reflexión del LLM sobre su propia respuesta
    # Por defecto es una cadena vacía ""
    reflection: str = ""
    
    # revised: indica si la respuesta necesita ser revisada (True) o está aprobada (False)
    # Por defecto es False (respuesta aprobada)
    revised: bool = False
    
    # attempts: contador de intentos de generación de respuesta
    # Por defecto es 0, se incrementa cada vez que se genera una respuesta
    attempts: int = 0

In [5]:
# -------------------------
# 3. Nodos (Funciones del Grafo)
# -------------------------

# a. Recuperar Documentos (Retrieve)
# Esta función recupera documentos relevantes del vector store basándose en la pregunta
def retrieve_docs(state: RAGReflectionState) -> RAGReflectionState:
    # Invoca al retriever con la pregunta del estado actual
    # Devuelve una lista de documentos similares semánticamente
    docs = retriever.invoke(state.question)
    
    # Retorna una copia actualizada del estado con los documentos recuperados
    # .model_copy(update={...}) crea una nueva instancia del estado con campos actualizados
    return state.model_copy(update={"retrieved_docs": docs})

# b. Generar Respuesta (Generate Answer)
# Esta función genera una respuesta usando el LLM basándose en los documentos recuperados
def generate_answer(state: RAGReflectionState) -> RAGReflectionState:
    
    # Concatena el contenido de todos los documentos recuperados con dos saltos de línea entre ellos
    # Esto crea un contexto unificado para el LLM
    context = "\n\n".join([doc.page_content for doc in state.retrieved_docs])
    
    # Construcción del prompt para el LLM
    # Incluye instrucciones, el contexto recuperado y la pregunta del usuario
    prompt = f"""
Usa el siguiente contexto para responder la pregunta:

Contexto:
{context}

Pregunta:
{state.question}
"""
    # Invoca al LLM con el prompt, extrae el contenido de la respuesta y elimina espacios en blanco
    answer = llm.invoke(prompt).content.strip()
    
    # Retorna el estado actualizado con la respuesta generada y el contador de intentos incrementado en 1
    return state.model_copy(update={"answer": answer, "attempts": state.attempts + 1})

In [6]:
# c. Auto-Reflexión (Self-Reflect)
# Esta función evalúa la calidad de la respuesta generada usando el mismo LLM como crítico
def reflect_on_answer(state: RAGReflectionState) -> RAGReflectionState:
    
    # Construcción del prompt de reflexión
    # Le pide al LLM que evalúe si la respuesta responde completamente la pregunta
    prompt = f"""
Reflexiona sobre la siguiente respuesta para ver si responde completamente la pregunta.
Indica SÍ si está completa y correcta, o NO con una explicación.

Pregunta: {state.question}

Respuesta: {state.answer}

Responde así:
Reflexión: SÍ o NO
Explicación: ...
"""
    # Invoca al LLM con el prompt de reflexión y obtiene el resultado
    result = llm.invoke(prompt).content
    
    # Verifica si la reflexión contiene "reflection: yes" (en minúsculas)
    # is_ok será True si el LLM aprobó la respuesta, False si necesita revisión
    is_ok = "reflection: yes" in result.lower()
    
    # Retorna el estado actualizado con:
    # - reflection: el texto completo de la evaluación del LLM
    # - revised: True si necesita revisión (not is_ok), False si está aprobada
    return state.model_copy(update={"reflection": result, "revised": not is_ok})

In [7]:
# d. Finalizador (Finalizer)
# Esta función marca el final del flujo de trabajo
# Simplemente retorna el estado sin modificaciones
def finalize(state: RAGReflectionState) -> RAGReflectionState:
    # Retorna el estado tal como está (sin cambios)
    return state

In [8]:
# -------------------------
# 4. Grafo Dirigido Acíclico (DAG) de LangGraph
# -------------------------

# Creación del constructor del grafo de estado usando la clase RAGReflectionState
# Este grafo define el flujo de trabajo completo del sistema RAG con auto-reflexión
builder = StateGraph(RAGReflectionState)

# Agregado de nodos al grafo
# Cada nodo es una función que procesa y transforma el estado

# Nodo "retriever": recupera documentos relevantes del vector store
builder.add_node("retriever", retrieve_docs)

# Nodo "responder": genera una respuesta usando el LLM basándose en los documentos recuperados
builder.add_node("responder", generate_answer)

# Nodo "reflector": evalúa la calidad de la respuesta generada (auto-crítica)
builder.add_node("reflector", reflect_on_answer)

# Nodo "done": finaliza el flujo de trabajo
builder.add_node("done", finalize)

# Establecer el punto de entrada del grafo (primer nodo a ejecutar)
# El flujo comienza en "retriever"
builder.set_entry_point("retriever")

# Definición de aristas (edges) - conexiones directas entre nodos

# Después de "retriever", siempre ir a "responder"
builder.add_edge("retriever", "responder")

# Después de "responder", siempre ir a "reflector"
builder.add_edge("responder", "reflector")

# Arista condicional después de "reflector"
# La función lambda decide el siguiente nodo basándose en el estado:
# - Si revised=False (respuesta aprobada) O attempts>=2 (máximo de intentos alcanzado) → ir a "done"
# - Si revised=True (necesita revisión) Y attempts<2 → volver a "retriever" para intentar de nuevo
builder.add_conditional_edges(
    "reflector",
    lambda s: "done" if not s.revised or s.attempts >= 2 else "retriever"
)

# Después de "done", ir a END (marcador especial que termina el grafo)
builder.add_edge("done", END)

# Compilación del grafo para hacerlo ejecutable
# .compile() valida la estructura y crea el grafo final
graph = builder.compile()

In [9]:
# -------------------------
# 5. Ejecutar el Agente
# -------------------------

# Verifica si este script se está ejecutando como programa principal (no importado como módulo)
if __name__ == "__main__":
    
    # Definición de la pregunta del usuario
    # Esta será la consulta que el sistema RAG intentará responder
    user_query = "¿Cuáles son las variantes de transformers en despliegues de producción?"
    
    # Creación del estado inicial con la pregunta del usuario
    # Los demás campos (retrieved_docs, answer, etc.) usarán sus valores por defecto
    init_state = RAGReflectionState(question=user_query)
    
    # Invocación del grafo con el estado inicial
    # .invoke() ejecuta todo el flujo de trabajo y retorna el estado final
    result = graph.invoke(init_state)

    # Impresión de la respuesta final generada por el sistema
    print("\n🧠 Respuesta Final:\n", result["answer"])
    
    # Impresión del log de reflexión (evaluación de la respuesta)
    print("\n🔁 Log de Reflexión:\n", result["reflection"])
    
    # Impresión del número total de intentos realizados
    print("🔄 Intentos Totales:", result["attempts"])


🧠 Respuesta Final:
 El contexto proporcionado no aborda específicamente las variantes de transformadores en despliegues de producción. Sin embargo, en el ámbito del aprendizaje automático y la inteligencia artificial, algunas variantes comunes de los transformadores que suelen utilizarse en despliegues de producción incluyen:

1. **BERT (Bidirectional Encoder Representations from Transformers):** Muy utilizado en tareas de procesamiento del lenguaje natural (NLP) como clasificación de texto, análisis de sentimiento y respuesta a preguntas.

2. **GPT (Generative Pre-trained Transformer):** Con versiones como GPT-2 y GPT-3, estos modelos son populares para generación de texto y aplicaciones de NLP que requieren creación de contenido.

3. **Transformer-XL:** Diseñado para manejar dependencias a largo plazo en tareas secuenciales, mejorando el rendimiento en comparación con transformers estándar.

4. **RoBERTa (A Robustly Optimized BERT Pretraining Approach):** Una versión optimizada de B